In [ ]:
import pandas as pd
import requests, re
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from bs4 import BeautifulSoup
import numpy as np
import urllib
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from fake_useragent import UserAgent
ua = UserAgent()
headers = {'user-agent':ua.firefox}

In [ ]:
# Initialising the required containers for product details
links = ["https://www.amazon.in/Samsung-Galaxy-Electric-Blue-128GB-Storage/dp/B085J1J32G/ref=sr_1_1_sspa?dchild=1&keywords=mobile&qid=1613815847&sr=8-1-spons&psc=1&spLa=ZW5jcnlwdGVkUXVhbGlmaWVyPUExUlZRV1FFRjJTVFdPJmVuY3J5cHRlZElkPUEwNTAxMDkwM0hLSUIzSlhYSFFYMSZlbmNyeXB0ZWRBZElkPUEwMDE5MDUwMTdNNTZZRzVPRTU0OCZ3aWRnZXROYW1lPXNwX2F0ZiZhY3Rpb249Y2xpY2tSZWRpcmVjdCZkb05vdExvZ0NsaWNrPXRydWU=",
        "https://www.amazon.in/Samsung-Galaxy-Storage-Additional-Exchange/dp/B07XBYLRVB/ref=pb_allspark_session_sims_desktop_5?pd_rd_w=RpChi&pf_rd_p=8375ff61-9bee-4cf7-9904-615da5a55bd5&pf_rd_r=W7D5BNN0P669AN2WV486&pd_rd_r=4644033c-d773-4023-aa56-151632749820&pd_rd_wg=IIxtM&pd_rd_i=B07XBYLRVB&psc=1",
        "https://www.amazon.in/dp/B07XG2PKVS/ref=sspa_dk_detail_1?psc=1&pd_rd_i=B07XG2PKVS&pd_rd_w=CHW5B&pf_rd_p=b9175453-ca9b-41ce-82bc-58f20ea9bb05&pd_rd_wg=mmwIa&pf_rd_r=NC956KRHC05B6X1M4KP8&pd_rd_r=14a40ec2-13d5-4d2d-b6f8-5a3a3b38e0bf&spLa=ZW5jcnlwdGVkUXVhbGlmaWVyPUEzNTRaMkFHSzZXOEFSJmVuY3J5cHRlZElkPUEwNTg2ODE5MUhMVUtXNkgyNVBXQiZlbmNyeXB0ZWRBZElkPUEwNjM5OTIxMTZNOERYR1dVWEJETSZ3aWRnZXROYW1lPXNwX2RldGFpbCZhY3Rpb249Y2xpY2tSZWRpcmVjdCZkb05vdExvZ0NsaWNrPXRydWU="]
names = []
availability = []
number_of_reviews = []
product_ratings = []
product_prices = []
technical_specs = []
related_products = []
product_reviews = []
list_of_product_images = []

In [ ]:
for link in links:
    r=requests.get(link, headers=headers,timeout=9)
    c=r.content
    soup=BeautifulSoup(c,"html.parser")
    
    # Product names
    #Product_name = soup.find("span",{"class","a-size-large product-title-word-break"}).text.strip()
    Product_name = soup.find("span", attrs={"id":'productTitle'}).text.strip()
    names.append(Product_name)
    
    # Product Availability
    available = soup.find("div", attrs={'id':'availability'}).find("span").string.strip()
    availability.append(available)
    
    # Product Review Counts
    review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()
    number_of_reviews.append(review_count)
    
    # Product Rating
    #rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
    product_ratings.append(rating)
    
    # Product Price
    price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip().replace("\xa0","")
    product_prices.append(price)
    
    # Technical Specs
    tech_specs = soup.find_all("table", attrs={'class':"a-keyvalue prodDetTable"})[0]   # Specs table
    features = tech_specs.find_all("th")
    specs = tech_specs.find_all("td")
    tech_specification = ""
    for i in len(specs):
        feauture = features[i].text.strip()
        spec = specs[i].text.strip()
        tech_specification += "* " + feauture + " : " + spec + "/n"
    technical_specs.append(tech_specification)
        
    # Related products
    relate = soup.find_all("div", attrs={'class':'a-carousel-col a-carousel-center'})
    related_items = relate.find_all("li")
    related_product = ""
    for product in related_items:
        related_product_name = product.find("div", attrs={'class':'sponsored-products-truncator-truncated'}).text.strip()
        related_product_price = product.find("span", attrs={'class':'a-size-medium a-color-price'}).text.strip().replace("\xa0","")
        related_product += "* " + related_product_name + " : " + related_product_price + "/n"
    related_products.append(related_product)
    
    # Reviews
    reviews = soup.find_all("a", attrs={'class':"a-size-base a-link-normal review-title a-color-base review-title-content a-text-bold"})
    reviews_bulleted = ""
    for review in reviews:
        reviews_bulleted += "* " + (review.text.strip()) + "/n"
    product_reviews.append(reviews_bulleted)
    time.sleep(30)

In [ ]:
# Starting Selenium Chrome Driver
driver = webdriver.Chrome(ChromeDriverManager().install())

# Defining url to image function
def url_to_image(url):
    #download the image, convert it to a NumPy array, and read
    resp = urllib.request.urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    #return the image array
    return image

# Extracting image links and then image arrays
for link in links:
    driver.get('https://www.amazon.in/Samsung-Galaxy-Electric-Blue-128GB-Storage/dp/B085J1J32G/ref=sr_1_1_sspa?dchild=1&keywords=mobile&qid=1613815847&sr=8-1-spons&psc=1&spLa=ZW5jcnlwdGVkUXVhbGlmaWVyPUExUlZRV1FFRjJTVFdPJmVuY3J5cHRlZElkPUEwNTAxMDkwM0hLSUIzSlhYSFFYMSZlbmNyeXB0ZWRBZElkPUEwMDE5MDUwMTdNNTZZRzVPRTU0OCZ3aWRnZXROYW1lPXNwX2F0ZiZhY3Rpb249Y2xpY2tSZWRpcmVjdCZkb05vdExvZ0NsaWNrPXRydWU=')
    action = ActionChains(driver)
    curser_points = ['//*[@id="a-autoid-8"]/span/input','//*[@id="a-autoid-10"]/span/input','//*[@id="a-autoid-11"]/span/input','//*[@id="a-autoid-12"]/span/input','//*[@id="a-autoid-13"]/span/input','//*[@id="a-autoid-14"]/span/input']
    image_links = []
    product_images = []
    
    # Performing Curser actions to focus on the product images and extracting the urls
    for point in curser_points:
        element = driver.find_element_by_xpath(point)
        action.move_to_element(element).perform()
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'lxml')
        image_path = soup.find_all('li', attrs={'class': lambda e: e.startswith('image item itemNo') if e else False})
        image_link = image_path[-1].find("img").attrs['src']
        image_links.append(image_link)
    
    # Getting image arrays from the image urls
    for img_url in image_links:
        img_array = url_to_image(url)
        product_images.append(img_array)
    list_of_product_images.append(product_images)
    time.sleep(30)

In [ ]:
products_df = pd.DataFrame(list(zip(names, availability, number_of_reviews, product_ratings, product_prices, technical_specs, related_products, product_reviews, list_of_product_images, links)), 
               columns =["Product Name", "Product Availability", "Number of Reviews", "Product Rating", "Product Price", "Technical Specifications", "Related Products", "Product Reviews", "Product Images as Arrays", "Amazon Product Url"])

In [ ]:
products_df.head()